# News Recommendation system

In [236]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import time 
import math
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords

In [237]:
#reading the dataset
df_ini = pd.read_csv("articles_data.csv.zip")

In [238]:
#dropping unnecessary columns
df_ini.drop(['Unnamed: 0','top_article','engagement_reaction_count','engagement_comment_count','engagement_share_count','engagement_comment_plugin_count'], axis=1, inplace=True)


In [239]:
#removing duplicates and resetting index
df = df_ini.drop_duplicates(subset='title', keep="first")
df = df.reset_index(drop=True)


In [240]:
#for converting text to vector
def text_to_vector(text):
    WORD = re.compile(r"\w+")
    words = WORD.findall(text)
    return Counter(words)

#for getting cosine simliarity between two sentences
def get_cosine(vector1, vector2):
    intersection = set(vector1.keys()) & set(vector2.keys())
    numerator = sum([vector1[x]*vector2[x] for x in intersection])
    sum1 = sum([vector1[x]*vector1[x] for x in list(vector1.keys())])
    sum2 = sum([vector2[x]*vector2[x] for x in list(vector2.keys())])
    denominator = math.sqrt(sum1)*math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator)/denominator

In [241]:
#TEXT PREPROCESSING

#changing all to lowercase
df['description'] = df['description'].str.lower()

In [242]:
#removing punctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    new_string = str(text).translate(str.maketrans('', '', string.punctuation))
    return new_string

for i in range(len(df['description'])):
    df['description'][i]=remove_punctuation(df['description'][i])


In [235]:
#removing stopwords from the sentence
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

for i in range(len(df['description'])):
    df['description'][i]=remove_stopwords(df['description'][i])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chgay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [225]:
#To recommend news based on cosine similarity
def recommend(query):
    list1=[]
    for i in range(len(headlines_list['description'])):
        vector1 = text_to_vector(query)
        vector2 = text_to_vector(str(headlines_list['description'][i]))
        list1.append(get_cosine(vector1, vector2)) #for getting the similarity
        
    headlines_list['score']= list1
    rslt_df = headlines_list.sort_values(by='score',ascending = False)

    final = list(rslt_df['title'])
    final_im = list(rslt_df['url_to_image'])
    final_des= list(rslt_df['description'])
    final_url= list(rslt_df['url']) #making a list of necessary columns

    recommendations_images = []
    recommendations_news=[]
    recommendations_descrip =[]
    recommendations_url=[] 

    t=0
    for k in rslt_df['title']:
        if(t< 6):
        recommendations_news.append(k)
        recommendations_images.append(final_im[t])
        recommendations_descrip.append(final_des[t])
        recommendations_url.append(final_url[t])
        t= t+1
        
    return recommendations_news,recommendations_images,recommendations_descrip,recommendations_url #returning the recommended news lists

In [226]:
import pickle

In [206]:
#serilaization
pickle.dump(df,open('headlines.pkl','wb'))